# 작업 1유형

## 데이터 로드

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/spotify/spotify.csv')
df

,title,artist,top genre,year released,added,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top year,artist type
0,STARSTRUKK (feat. Katy Perry),3OH!3,dance pop,2009.0,2022‑02‑17,140.0,81.0,61.0,-6.0,23.0,23.0,203.0,0.0,6.0,70.0,2010.0,Duo
1,My First Kiss (feat. Ke$ha),3OH!3,dance pop,2010.0,2022‑02‑17,138.0,89.0,68.0,-4.0,36.0,83.0,192.0,1.0,8.0,68.0,2010.0,Duo
2,I Need A Dollar,Aloe Blacc,pop soul,2010.0,2022‑02‑17,95.0,48.0,84.0,-7.0,9.0,96.0,243.0,20.0,3.0,72.0,2010.0,Solo
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,atl hip hop,2010.0,2022‑02‑17,93.0,87.0,66.0,-4.0,4.0,38.0,180.0,11.0,12.0,80.0,2010.0,Solo
4,Nothin' on You (feat. Bruno Mars),B.o.B,atl hip hop,2010.0,2022‑02‑17,104.0,85.0,69.0,-6.0,9.0,74.0,268.0,39.0,5.0,79.0,2010.0,Solo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,Strike a Pose (feat. Aitch),Young T & Bugsey,afroswing,2019.0,2020‑08‑20,138.0,58.0,53.0,-6.0,10.0,59.0,214.0,1.0,10.0,67.0,2019.0,Duo
999,The London (feat. J. Cole & Travis Scott),Young Thug,atl hip hop,2019.0,2020‑06‑22,98.0,59.0,80.0,-7.0,13.0,18.0,200.0,2.0,15.0,75.0,2019.0,Solo
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 각 년도별 1-100위 랭킹을 나타내는 rank 컬럼을 만들고 매년도 1위의 bpm 컬럼의 평균값 

In [10]:
df.dropna(inplace=True)
df['rank'] = list(range(1,101))*10
print(df.loc[df['rank']==1].bpm.mean())

125.6


## 2015년도에 가장 많은 top100 곡을 올린 artist

In [19]:
print(df.loc[df['top year'] == 2015].artist.value_counts().index[0:2])

Index(['Taylor Swift', 'The Weeknd'], dtype='object', name='artist')


## 년도별 Rank 값이 1-10위 까지의 곡들 중 두번째로 많은 top genre

In [28]:
print(df.loc[df['rank'] <= 10]['top genre'].value_counts().index[1:3])

Index(['atl hip hop', 'british soul'], dtype='object', name='top genre')


## 피처링의 경우 title에 표시됨, 피처링을 가장 많이 해준 가수

In [40]:
def featuring(row):
    if '(feat.' in row:
        return row.split('(feat.')[1][:-1]
df['featuring'] = df['title'].apply(featuring)
print(df.featuring.value_counts().sort_values(ascending=False).index[0])

 Bruno Mars


## top year 년도를 기준으로 발매일과 top100 에 진입한 일자가 다른 곡의 숫자가 가장 많은 년도

In [48]:
print(df.loc[df['year released'] != df['top year']].value_counts('top year').index[0])

2016.0


## artist 컬럼의 값에 대소문자 상관없이 q 단어가 들어가는 아티스트

In [61]:
print(len(df.loc[df['artist'].str.lower().str.contains('q')].artist.unique()))

6


## 년도 상관없이 전체 데이터에서 1-50위와 51-100위간의 dur 컬럼의 평균값 차이

In [66]:
print(df.loc[df['rank'] <= 50].dur.mean() - df.loc[df['rank'] > 50].dur.mean())

0.896000000000015


## title을 띄어쓰기 단어로 구분 했을 때 가장 많이 나온 단어

In [81]:
print(df.title.str.split('\(feat.').str[0].str.split().explode().str.lower().value_counts().index[0])

the


## 년도별 nrgy 값의 평균값을 구할 때 최대 평균값과 최소 평균값의 차이

In [86]:
print(df.groupby('top year').nrgy.mean().sort_values().values[-1] - df.groupby('top year').nrgy.mean().sort_values().values[0])

13.860000000000007


## artist 중 artist type을 여러 개 가지고 있는 artist

In [95]:
print(df.groupby(['artist','artist type']).size().reset_index().artist.value_counts().loc[df.groupby(['artist','artist type']).size().reset_index().artist.value_counts() > 1].index)

Index(['Rudimental'], dtype='object', name='artist')


# 작업 2유형

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score 
from sklearn.ensemble import RandomForestClassifier 
#데이터 로드
x = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_train.csv")
y = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/y_train.csv")
test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_test.csv")

x.drop('ID',axis=1,inplace= True)
y.drop('ID',axis=1,inplace= True)
test_x = test.drop('ID',axis=1)

xtr,xte,ytr,yte = train_test_split(x,y,test_size= 0.3, random_state = 23)

RFC = RandomForestClassifier(random_state = 23)
RFC.fit(xtr,ytr)
pred = RFC.predict(xte)
pred_proba = RFC.predict_proba(xte)[:,1]

acc = accuracy_score(yte,pred)
f1 = f1_score(yte,pred)
recall = recall_score(yte,pred)
precision = precision_score(yte,pred)
auc = roc_auc_score(yte,pred_proba)
print(acc)
print(f1)
print(recall)
print(precision)
print(auc)

test_pred = RFC.predict(test_x)
test_pred_proba = RFC.predict_proba(test_x)[:,1]
test['pose'] = test_pred_proba
test[['ID','pose']].to_csv('111.csv',index=False)

/usr/local/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.9949820788530466
0.9949312092686459
0.9913419913419913
0.998546511627907
0.999909555465111
